# Graph theory in math bio: food webs

<a href="https://drive.google.com/file/d/1d20M8r1oJoPLB8zZBAM-7WAVM6rrVFJK/view?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook lets you explore a small biological food web using graph theory.


Firstly we need to define our graph in python. We do this in a dictionary.

In [ ]:
from collections import deque, defaultdict

edges = [
    ("Grass","SmallFish"),
    ("Algae","Zooplankton"),
    ("Zooplankton","SmallFish"),
    ("SmallFish","BigFish"),
    ("SmallFish","Crab"),
    ("Crab","Bird"),
    ("BigFish","Seal"),
    ("Seal","Shark"),
    ("Jellyfish","Shark"),
    ("Zooplankton","Jellyfish"),
    ("Bird","Shark"),
]

G = defaultdict(list)
for a,b in edges:
    G[a].append(b)
    G[b].append(a)

G

defaultdict(list,
            {'Grass': ['SmallFish'],
             'SmallFish': ['Grass', 'Zooplankton', 'BigFish', 'Crab'],
             'Algae': ['Zooplankton'],
             'Zooplankton': ['Algae', 'SmallFish', 'Jellyfish'],
             'BigFish': ['SmallFish', 'Seal'],
             'Crab': ['SmallFish', 'Bird'],
             'Bird': ['Crab', 'Shark'],
             'Seal': ['BigFish', 'Shark'],
             'Shark': ['Seal', 'Jellyfish', 'Bird'],
             'Jellyfish': ['Shark', 'Zooplankton']})

This code finds the degrees of each node (how many things they have connected to them):

In [ ]:
print("Degrees:")
for node in sorted(G):
    print(node, len(G[node]))

Degrees:
Algae 1
BigFish 2
Bird 2
Crab 2
Grass 1
Jellyfish 2
Seal 2
Shark 3
SmallFish 4
Zooplankton 3


This code finds the shortest path between two components:

In [ ]:
def bfs_shortest_path(start, goal):
    q = deque([[start]])
    seen = {start}
    while q:
        path = q.popleft()
        node = path[-1]
        if node == goal:
            return path
        for neigh in G[node]:
            if neigh not in seen:
                seen.add(neigh)
                q.append(path + [neigh])
    return None

bfs_shortest_path("Algae", "Shark")

['Algae', 'Zooplankton', 'Jellyfish', 'Shark']

This code checks if all the graph components are connected. We can then use this to see if removing any nodes results in the graph being disconnected.

In [ ]:
def connected_components(graph):
    seen = set()
    comps = []
    for n in graph:
        if n not in seen:
            comp = []
            stack = [n]
            seen.add(n)
            while stack:
                u = stack.pop()
                comp.append(u)
                for v in graph[u]:
                    if v not in seen:
                        seen.add(v)
                        stack.append(v)
            comps.append(comp)
    return comps

orig_comps = connected_components(G)
print("\nOriginal component count:", len(orig_comps))

vulnerable = []
for node in list(G):
    G2 = {u: [v for v in G[u] if v!=node] for u in G if u!=node}
    comps = connected_components(G2)
    if len(comps) > len(orig_comps):
        vulnerable.append(node)
print("Articulation-like nodes (single removal increases components):", vulnerable)


Original component count: 1
Articulation-like nodes (single removal increases components): ['SmallFish', 'Zooplankton']


# Challenges:
Can you adapt the code above to explore these other biological networks?


# 1: Miniature sensory–motor reflex circuit

Nodes = neurons

Edges = synapses (directed)

* TouchSensor → Interneuron1
* TouchSensor → Interneuron2
* Interneuron1 → MotorNeuronA
* Interneuron2 → MotorNeuronB
* Interneuron1 → Interneuron2
* MotorNeuronA → InhibitoryNeuron
* InhibitoryNeuron → MotorNeuronB

# 2: Mini oxidative phosphorylation mini-network

Nodes = proteins/complexes

Edges = physical or functional interaction (not directed)

* NADH_DH — CoQ
* CoQ — ComplexIII
* ComplexIII — CytochromeC
* CytochromeC — ComplexIV
* ComplexIV — ATP_Synthase
* ComplexIII — ATP_Synthase
* CoQ — ComplexIV

Which proteins are hubs?

What happens if CoQ is removed?

Identify parallel paths (two paths that get to the same place) → redundancy/resilience in biological systems.

# 3: Simple contact network for outbreak modelling

Nodes = individuals (or households, or towns)

Edges = disease-relevant contact

Can be undirected (close contact) or directed (infection chain).

* Alex — Ben
* Alex — Cara
* Ben — Dana
* Cara — Elif
* Cara — Ben
* Elif — Farah
* Dana — George
* George — Ben
* Farah — Henry

Who are super-spreaders? (High-degree nodes: Alex, Cara, Ben)

What’s the shortest infection path from Alex to Henry?

If Cara isolates, how many infections does that prevent?

Identify clusters / components (e.g. groups with dense internal edges).